# pyslammer demo: batch simulations
This notebook shows an example use case of pyslammer for running batch simulations.
## Setup

Assumes pyslammer is installed in the current python environment. 
*Documentation on installation pending*, but for now, you can install it using pip:

```bash
pip install pyslammer
```

Import pyslammer using:

```python
import pyslammer as slam
```

Additional Python libraries, such as `matplotlib` may also be useful.

In [ ]:
%matplotlib widget

In [ ]:
import pyslammer as slam
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
kys = np.linspace(0.1,0.7,100)
histories = slam.sample_ground_motions()
output = {}

# Single simulation
All the sample ground motions that come with pyslammer have now been loaded into `histories` as a dictionary of name:time_history pairs.
The cells below print the list of available ground motions and then run a single simulation on one example (`Chi-Chi_1999_TCU068-090`).

In [ ]:
motion_options = [print(history) for history in histories.keys()]

In [ ]:
# single simulation
ky = 0.2
motion = "Imperial_Valley_1979_BCR-230"

single_result = slam.RigidAnalysis(histories[motion].accel, histories[motion].dt, ky)

plt.close('all')
fig, axs = single_result.sliding_block_plot()
plt.show()

# Batch Simulations
The code below evaluates all combinations of $k_y$ contained in `kys` and every motion in `histories`.
Some key features (motion, ky, max displacement, and displacement time histories) are stored in a dictionary, which 
is then converted to a pandas dataframe.

In [ ]:
run = 0
for ky in kys:
    for motion, hist in histories.items():
        result = slam.RigidAnalysis(histories[motion].accel, histories[motion].dt, ky)
        output[run] = {"motion":motion, "ky":ky, "d_max": result.max_sliding_disp, "dt":result.dt, "disp": result
        .sliding_disp}
        run += 1

In [ ]:
# convert the output to a pandas dataframe
df = pd.DataFrame.from_dict(output,orient='index')

## Comparing multiple motions
The results of the analyses can be plotted to show trends in total accumulated displacement with $k_y$ for each ground motion in the sample ground motion suite.

In [ ]:
plt.close('all')
fig, ax = plt.subplots()
fig.set_size_inches(10,6)   
for key, grp in df.groupby(['motion']):
    ax.scatter(grp["ky"], grp["d_max"], label=key[0], alpha=0.5)
ax.legend(loc='upper left', bbox_to_anchor=(1, 1))
ax.set_yscale('log')
ax.set_ylim(1e-3,1e1)
ax.set_xlabel('Yield Acceleration (g)')
ax.set_ylabel('Maximum Displacement (m)')
ax.set_title('Maximum Displacement vs. Yield Acceleration')

plt.tight_layout()
plt.show()

## Variation in a single motion's time history  
Alternatively, for any given motion, the displacement time histories for different values of $k_y$. The data for all the other motions could get plotted, too, but there's not really much to compare at that point because the plots get too messy.


In [ ]:
df

In [ ]:
import matplotlib.cm as cm
from matplotlib.colors import LogNorm

motion = "Imperial_Valley_1979_BCR-230"

plt.close('all')

# Create a figure and axes
fig, ax = plt.subplots(figsize=(6, 4))

# Create a color map
cmap = plt.colormaps['Spectral']#cm.get_cmap('viridis')
norm = LogNorm(df['ky'].min(), df['ky'].max())

dt = df[df["motion"]==motion].iloc[0]['dt']
npts = df[df["motion"]==motion].iloc[0]['disp'].shape[0]
time = np.linspace(0, dt*npts, npts)

for index, row in df[df["motion"]==motion].iterrows():
    color = cmap(norm(row['ky']))
    ax.plot(time, row['disp'], color=color)

# Add a color bar
sm = cm.ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])
cbar = fig.colorbar(sm, ax=ax, label='ky')

# Set the colorbar ticks and labels
ticks = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7]
cbar.set_ticks(ticks)
cbar.set_ticklabels([f'{tick:.2f}' for tick in ticks]) 

# Set the x-axis and y-axis labels
ax.set_xlabel('Time')
ax.set_ylabel('Displacement')
ax.set_title(motion)
plt.show()